## Assignments for internship

### Question 1 (Fully Connected Neural Network): 
Consider that you have a neural network with two hidden layers and let X be the input, W1, W2, W3 be the weights of the hidden layers, b1, b2, b3 be the corresponding biases and y be the output. Implement the forward pass and backward pass (you can assume any loss function and activation function). You don’t have to test your code on any data. We will just check the implementation and ask questions based on that (If possible vectorize your code using numpy). 


In [40]:
# Initialize the network 
'''
n_inputs = no. of input neurons
n_hidden = a list of size n_hidden_layers which contains no. of neurons in each hidden layer
n_outputs = no. of output neurons
bias = bias for each neuron in the layer should be specified
'''
network = list()

def create_hidden_layer(n_inputs, n_hidden, bias):
    hidden_layer = [{'weights':[random() for j in range(n_inputs)], 'bias' : bias[i]} for i in range(n_hidden)]
    network.append(hidden_layer)

def create_output_layer(n_hidden, n_outputs, bias):
    output_layer = [{'weights':[random() for j in range(n_hidden)], 'bias' : bias[i]} for i in range(n_outputs)]
    network.append(output_layer)
    

In [41]:
from random import seed
from random import random

# example
seed(1)
create_hidden_layer(2,3,[10,11,12])
create_hidden_layer(3,4,[13,14,15,16])
create_output_layer(4,1,[9])

print 'NETWORK : '
for layer in network:
    print layer
    print '\n'

NETWORK : 
[{'bias': 10, 'weights': [0.13436424411240122, 0.8474337369372327]}, {'bias': 11, 'weights': [0.763774618976614, 0.2550690257394217]}, {'bias': 12, 'weights': [0.49543508709194095, 0.4494910647887381]}]


[{'bias': 13, 'weights': [0.651592972722763, 0.7887233511355132, 0.0938595867742349]}, {'bias': 14, 'weights': [0.02834747652200631, 0.8357651039198697, 0.43276706790505337]}, {'bias': 15, 'weights': [0.762280082457942, 0.0021060533511106927, 0.4453871940548014]}, {'bias': 16, 'weights': [0.7215400323407826, 0.22876222127045265, 0.9452706955539223]}]


[{'bias': 9, 'weights': [0.9014274576114836, 0.030589983033553536, 0.0254458609934608, 0.5414124727934966]}]




### 2. Forward Propagate
We can calculate an output from a neural network by propagating an input signal through each layer until the output layer outputs its values.

We call this forward-propagation.

It is the technique we will need to generate predictions during training that will need to be corrected, and it is the method we will need after the network is trained to make predictions on new data.

We can break forward propagation down into three parts:

- Neuron Activation.
- Neuron Transfer.
- Forward Propagation.

##### Neuron Activation

It is calculated as the weighted sum of inputs and then adding it to the bias unit of that layer

In [12]:
# Calculate neuron activation 
def activate(weights, inputs, bias):
    activation = bias
    for i in range(len(weights)):
        activation += weights[i] * inputs[i]
    return activation

##### Neuron Transfer

Let's use the traditional transfer function : the sigmoid function for this job 

In [13]:
# Transfer neuron activation
import math
def transfer(activation):
    return 1.0 / (1.0 + math.exp(-activation))

##### Forward Propagation

In [52]:
# Forward propagate input to a network output
def forward_propagate(network, row):
    inputs = row
    i = 0
    for layer in network:
        new_inputs = []
        print '\nnew layer'
        for neuron in range(len(layer)):
            print 'neuron' + str(i)
            i+=1
            print layer[neuron]['bias']
            activation = activate(layer[neuron]['weights'], inputs, layer[neuron]['bias'])
            act = transfer(activation)
            layer[neuron]['output'] = act
            new_inputs.append(act)
        inputs = new_inputs
        i=0
    return inputs

In [54]:
network = [[ {'weights': [0.13436424411240122, 0.8474337369372327],'bias' : 0.763774618976614}],
            [{'weights': [0.2550690257394217], 'bias' : 0.49543508709194095}, 
            {'weights': [0.4494910647887381], 'bias' : 0.651592972722763}]]
row = [1, 0, None]
output = forward_propagate(network, row)
print(output)



new layer
neuron0
0.763774618977

new layer
neuron0
0.495435087092
neuron1
0.651592972723
[0.6629970129852887, 0.7253160725279748]


### 3. Back prop

Error is calculated between the expected outputs and the outputs forward propagated from the network. These errors are then propagated backward through the network from the output layer to the hidden layer, assigning blame for the error and updating weights as they go.

##### Transfer Derivative

In [55]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
    return output * (1.0 - output)

##### Error Backprop

In [112]:
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != (len(network)-1):
            for j in range(len(layer)): # for 1 neuron in present layer
                error = 0.0
                present_neuron = layer[j]
                for prev_neuron in network[i+1]:
                    error += prev_neuron['weights'][j] * prev_neuron['delta'] 
                    error +=  prev_neuron['bias'] * prev_neuron['delta']
                    
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])
        
#         print network[i]
#         print '\n'

In [113]:
network = [[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327], 'bias' : 0.763774618976614}],
            [{'output': 0.6213859615555266, 'weights': [0.2550690257394217], 'bias' : 0.49543508709194095}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381], 'bias' : 0.651592972722763}]]
expected = [0, 1]
backward_propagate_error(network, expected)
for layer in network:
    print(layer)

[{'output': 0.7105668883115941, 'bias': 0.763774618976614, 'weights': [0.13436424411240122, 0.8474337369372327], 'delta': -0.005088768681049473}]
[{'output': 0.6213859615555266, 'bias': 0.49543508709194095, 'weights': [0.2550690257394217], 'delta': -0.14619064683582808}, {'output': 0.6573693455986976, 'bias': 0.651592972722763, 'weights': [0.4494910647887381], 'delta': 0.0771723774346327}]


### 4. Update Weights

In [ ]:
# weight = weight + learning_rate * error * input
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']